In [11]:
import cv2
import gc
import os,sys, os.path
from os import listdir
import numpy as np
import pandas as pd
import glob
import natsort
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import image
gc.collect()

7

In [2]:
labels_path = Path("train_apparel/train.csv")
labels = []
lines = labels_path.read_text().strip().splitlines()[1:]

In [3]:
for line in lines:
    line = line.split(',')
    line = [x.strip(' \n\t\r') for x in line]
    line[0], line[1] = int(line[0]), int(line[1])
    line = tuple(line)
    labels.append(line[1])

In [4]:
labels[:5]

[9, 0, 0, 3, 0]

In [5]:
# label_dict = {0 :'T-shirt/top', 1:'Trouser', 2 :'Pullover', 3 :'Dress', 4 :'Coat', 5 :'Sandal', 
#               6 :'Shirt', 7:'Sneaker', 8 :'Bag', 9 :'Ankle boot'}

In [6]:
train_images = []
for img in natsort.natsorted(os.listdir("train_apparel/train/")):
    train_images.append(img)

In [7]:
test_images = []
for img in natsort.natsorted(os.listdir("test_apparel/test/")):
    test_images.append(img)

### Checking if any image is corrupted

In [8]:
# for file in train_images:
#     try:
#         img = Image.open("train_apparel/train/" +file)
#         img.verify()
#     except (IOError, SyntaxError) as e:
#         print('Bad file:', file)   

### Converting image from png to array

In [9]:
train_data = []
for i in train_images:
    ima1 = image.imread("train_apparel/train/" + i)
    train_data.append(ima1[:,:,0:3])

In [10]:
train_data = np.array(train_data)

MemoryError: 

In [ ]:
test_data = []
for i in test_images:
    ima1 = image.imread("test_apparel/test/" + i)
    test_data.append(ima1[:,:,0:3])

In [ ]:
test_data = np.array(test_data)

### CNN

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.utils import np_utils
from keras.preprocessing import sequence

In [ ]:
y_train = np_utils.to_categorical(labels, 10) 

In [33]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu',data_format= "channels_last",input_shape= (28,28,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 128)       3584      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               800500    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5010      
Total para

In [38]:
model.fit(train_data, y_train, epochs = 40, batch_size= 500,  verbose=1,  validation_split=0.15)

Train on 51000 samples, validate on 9000 samples
Epoch 1/40
51000/51000 [==============================] - 63s 1ms/step - loss: 0.0648 - acc: 0.9769 - val_loss: 0.3274 - val_acc: 0.9086
Epoch 2/40
51000/51000 [==============================] - 60s 1ms/step - loss: 0.0654 - acc: 0.9771 - val_loss: 0.3110 - val_acc: 0.9097
Epoch 3/40
51000/51000 [==============================] - 62s 1ms/step - loss: 0.0576 - acc: 0.9796 - val_loss: 0.3024 - val_acc: 0.9186
Epoch 4/40
51000/51000 [==============================] - 61s 1ms/step - loss: 0.0468 - acc: 0.9837 - val_loss: 0.3329 - val_acc: 0.9097
Epoch 5/40
51000/51000 [==============================] - 60s 1ms/step - loss: 0.0432 - acc: 0.9857 - val_loss: 0.3397 - val_acc: 0.9127
Epoch 6/40
51000/51000 [==============================] - 59s 1ms/step - loss: 0.0376 - acc: 0.9869 - val_loss: 0.3449 - val_acc: 0.9152
Epoch 7/40
51000/51000 [==============================] - 62s 1ms/step - loss: 0.0350 - acc: 0.9879 - val_loss: 0.3629 - val_acc:

In [39]:
y_predict = model.predict_classes(test_data)

In [40]:
y_predict

array([9, 2, 1, ..., 8, 1, 5], dtype=int64)

In [41]:
test_csv = pd.read_csv('test_apparel/test.csv')

In [42]:
test_csv['label']  = y_predict

In [43]:
test_csv.to_csv('final_submission.csv')